In [7]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
from dimod import ExactSolver
from dwave.system.composites import EmbeddingComposite
import dwavebinarycsp as dbc


# 0. Connect to AWS

bucket = 'amazon-braket-7481abb5f17b'
prefix = 'dwave'
s3_path = (bucket, prefix)

device = AwsDevice('arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')

In [6]:
# 1. Express problem as CSP with boolean logic
## Problem: x3 = x1 ∧ x2

and_gate = dbc.factories.and_gate(['x1', 'x2', 'x3'])
csp = dbc.ConstraintSatisfactionProblem('BINARY')
csp.add_constraint(and_gate)

csp.check({'x1': 1, 'x2': 1, 'x3': 1})  # confirm boolean expression performs as intended

True

In [5]:
# 2. Convert CSP to a BQM

bqm = dbc.stitch(csp)
bqm.remove_offset()

print('Coefficients')
print("LINEAR:    \t{}".format({key: round(val, 2) for key, val in bqm.linear.items()}))
print("QUADRATIC: \t{}".format({key: round(val, 2) for key, val in bqm.quadratic.items()}))

Coefficients
LINEAR:    	{'x1': -0.0, 'x2': -0.0, 'x3': 6.0}
QUADRATIC: 	{('x1', 'x2'): 2.0, ('x1', 'x3'): -4.0, ('x2', 'x3'): -4.0}


In [8]:
# 3. Solve by Minimization

solver = ExactSolver()
solution = solver.sample(bqm)

print(solution)

  x1 x2 x3 energy num_oc.
5  1  1  1   -0.0       1
3  0  1  0   -0.0       1
1  1  0  0   -0.0       1
0  0  0  0    0.0       1
6  1  0  1    2.0       1
4  0  1  1    2.0       1
2  1  1  0    2.0       1
7  0  0  1    6.0       1
['BINARY', 8 rows, 8 samples, 3 variables]
